pip install git+https://github.com/huggingface/transformers.git

# Obtaining a Dataset
For this tutorial, they've decided the data should be in the format ChatML (Explore more into what ChatML is later)
They illustrate multiple ways of obtaining a dataset
1. download a preprocessed data set, OpenAssistant dataset is already in the ChatML format and does not need any preprocessing on our end
2. Download a dataset thats not in our format. This will need to be inspected and then changed to our format. See how in article
3. Take a transcript and format into ChatML format. See how in article
4. Take raw text and convert to ChatML format

This list goes from least work to most work
I will just use the easiest option.

!pip install -q -U bitsandbytes\
!pip install -q -U git+https://github.com/huggingface/transformers.git\
!pip install -q -U git+https://github.com/huggingface/peft.git\
!pip install -q -U git+https://github.com/huggingface/accelerate.git\


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [15]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]



In [18]:

batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

batch["labels"] = torch.tensor([1, 1])

optimizer = torch.optim.AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

In [20]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")

In [21]:
raw_train_dataset = raw_datasets["train"]
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])
inputs = tokenizer("This is the first sentence.", "This is the second one.")

In [22]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map: 100%|██████████| 408/408 [00:00<00:00, 1376.90 examples/s]


In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
batch = data_collator(samples)



In [30]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.device("cuda")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [29]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

TypeError: Trainer.__init__() got an unexpected keyword argument 'load_in_4bit'